# ДИПЛОМНЫЙ ПРОЕКТ "Модель прогнозирования стоимости жилья для агенства недвижимости"

<span style=color:green>Задача</span> - разработать модель, которая позволила бы агенству недвижимости обойти конкурентов по скорости и качеству сделок

## 1. ПОДГРУЗКА ДАННЫХ И НЕОБХОДИМЫХ БИБЛИОТЕК

In [99]:
import numpy as np
import pandas as pd
import re
from geopy.geocoders import Nominatim
from fake_useragent import UserAgent
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
from geopy.exc import GeocoderServiceError

In [100]:
df = pd.read_csv('data/data.csv/data.csv')

## 2. ЗНАКОМСТВО С ДАННЫМИ

In [101]:
df.head(5)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

**Описание данных:**  
* <span style=background-color:#D3D3D3>'status'</span> — статус продажи;  
* <span style=background-color:#D3D3D3>'private pool'</span> и <span style=background-color:#D3D3D3>'PrivatePool'</span> — наличие собственного бассейна;  
* <span style=background-color:#D3D3D3>'propertyType'</span> — тип объекта недвижимости;  
* <span style=background-color:#D3D3D3>'street'</span> — адрес объекта;  
* <span style=background-color:#D3D3D3>'baths'</span> — количество ванных комнат;  
* <span style=background-color:#D3D3D3>'homeFacts'</span> — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);  
* <span style=background-color:#D3D3D3>'fireplace'</span> — наличие камина;  
* <span style=background-color:#D3D3D3>'city'</span> — город;  
* <span style=background-color:#D3D3D3>'schools'</span> — сведения о школах в районе;  
* <span style=background-color:#D3D3D3>'sqft'</span> — площадь в футах;  
* <span style=background-color:#D3D3D3>'zipcode'</span> — почтовый индекс;  
* <span style=background-color:#D3D3D3>'beds'</span> — количество спален;  
* <span style=background-color:#D3D3D3>'state'</span> — штат;  
* <span style=background-color:#D3D3D3>'stories'</span> — количество этажей;  
* <span style=background-color:#D3D3D3>'mls-id'</span> и <span style=background-color:#D3D3D3>'MlsId'</span> — идентификатор MLS (Multiple Listing Service, система мультилистинга);  
* <span style=background-color:#D3D3D3>'target'</span> — цена объекта недвижимости (**целевой признак, который необходимо спрогнозировать**).

**Важные сведения о данных:**  
1. Признаки <span style=background-color:#D3D3D3>'homeFacts'</span> и <span style=background-color:#D3D3D3>'schools'</span> представляют собой словари и содержат несколько типов сведений. Необходимо десериализовать содержимое этих признаков. Если в нём есть полезные данные, необходимо создать из них новые признаки.

2. Типы жилья (может пригодиться при обработке признака <span style=background-color:#D3D3D3>propertyType</span>): 
    * **apartment** — съёмная квартира (нельзя купить); 
    * **condo** — кондоминиум (можно купить); 
    * **co-op** — квартира в жилищном кооперативе;
    * **single-family (detached, tiny home)** — односемейный (отдельный, маленький) дом;
    * **townhome, townhouse** — таунхаус;

3. Основные характеристики  жилья (может пригодиться при
обработке признака <span style=background-color:#D3D3D3>propertyType</span>): 
    * **cape cod** — черепичная крыша, облицовка деревянным сайдингом, дверь в центре здания, окна по бокам, один-два этажа; 
    * **colonial home** — минимум два этажа, симметрия, лестница в центре здания, строгий внешний вид; 
    * **contemporary** — чистые, простые линии, нейтральные цвета, натуральные текстуры;
    * **cottage** — небольшая уютная веранда, небольшие жилые помещения;
    * **craftsman** — деревянные элементы ручной работы, выступающие балки, большие колонны;
    * **Greek revival** — большие белые колонны, украшения в греческом стиле, нарядный вход;
    * **farmhouse** — прямоугольная планировка, большое крыльцо, амбарная крыша;
    * **French country** — влияние прованса, облицовка камнем, состаренный вид;
    * **Mediterranean** — белая штукатурка, дерево и камень в тёплых тонах, черепичные крыши, элементы испанских и итальянских вилл;
    * **midcentury modern** — сочетание плавных природных и строгих геометрических линий, гладкость, лаконичность, большие окна;
    * **ranch** — один этаж, низкая крыша, открытые жилые помещения, задний двор;
    * **split-level** — жилые помещения разделены короткими лестничными пролётами;
    * **Tudor** — деревянные балки, каменная кладка, асимметричная крутая двускатная крыша;
    * **Victorian (Queen Anne Houses)** — два-три и более этажей, богатый декор, крутая двускатная крыша, небольшие башенки, яркий фасад;
    * **European Houses style** — кирпич или камень, высокая крутая крыша, высокие окна, часто со ставнями, традиционные декоративные детали (фронтоны, замковые камни);
    * **log home** — деревянный дом.

4. Некоторые жаргонные сокращения — для исправления дублирования (может пригодиться при обработке признака <span style=background-color:#D3D3D3>propertyType</span>):
    * **manufactured home (house)** = **mobile** = **prefab** = **modular**; 
    * **mobile** = **mo2 le** = **mo2le**; 
    * **cabin** = **ca2 n** = **ca2n**; 
    * **midcentury** = **mid century** = **mid-century**;
    * **single family** = **single-family home**.
    
5. Некоторые термины, используемые в сфере купли-продажи недвижимости:
    * **status** — статус, состояние. 
    * **estate** — объект недвижимости. 
    * **foreclosed** (**foreclosure**) — «лишён права выкупа». *Это процесс, когда недвижимость переходит к кредитору из-за неуплаты ипотеки заёмщиком. Продажа foreclosure-недвижимости отличается от традиционных продаж.*
    * **pre foreclosure** (**pre foreclosure auction**) — «до выкупа», «аукцион до выкупа».
    * **under contract showing** (**under contract show**, **under contract**, **active under contract**) — предложение о покупке уже сделано, но не принято продавцом окончательно, и недвижимость продолжают показывать другим потенциальным покупателям.
    * **under contract backups**, **active backup**, **backup contract** — предложение о покупке уже сделано и принято продавцом, однако он открыт для резервных предложений. 
    * **contingency** — дополнительные обстоятельства (критерии, которым должен соответствовать продавец/покупатель до завершения сделки).
    * **contingency contract** — контракт на случай возникновения дополнительных обстоятельств.
    * **active contingency** — сделка уже практически состоялась, однако продавцу/покупателю для её завершения необходимо выполнить какие-то условия.
    * **insp inspection contingency** — ситуация, когда покупатель может официально проинспектировать объект недвижимости в определённый период времени.
    * **pending escape clause** — договор, предполагающий свободный выход сторон из него.
    * **pending backup wanted** — договор уже заключен, однако продавец хочет продолжать показывать недвижимость другим потенциальным покупателям и принимать резервные предложения.
    * **pending take backups** — условие, предполагающее, что продавец может принимать резерные предложения, если сделка с текущим покупателем не состоится.
    * **pending continue show** — договор уже заключен, однако продавец хочет продолжать показывать недвижимость другим потенциальным покупателям на случай, если текущая сделка не состоится.
    * **pending inspection** — «в ожидании инспекции».
    * **due diligence period** — период, в течение которого покупатель имеет право проинспектировать недвижимость и изучить всю важную для заключения сделки информацию.
    * **activated** — «активен», открыт для покупки.
    * **active with contract** — продавец уже принял предложение о покупке, однако сделка ещё не закрыта.
    * **active with offer** — продавец принял предложение о покупке, однако ищет резервные предложения на случай, если сделка с текущим покупателем не состоится.
    * **active auction**, **auction active**, **auction** — продажа недвижимости с помощью аукциона.
    * **cooling-off period** — период, в течение которого покупатель может расторгнуть договор после завершения сделки.

## 3. ОЧИСТКА ДАННЫХ

Посмотрим на количество дублирующих наблюдения в данных:

In [103]:
dubl = df.duplicated().sum()
print(f'В данных содержится {dubl} дубликатов')

В данных содержится 50 дубликатов


Удалим дубликаты:

In [104]:
df = df.drop_duplicates(ignore_index=True)

Посмотрим на количество пропусков в признаках:

In [105]:
# общщее колличество пропусков по каждому признаку
missings = df.isnull().sum()
missings

status           39917
private pool    372954
propertyType     34733
street               2
baths           106308
homeFacts            0
fireplace       274023
city                34
schools              0
sqft             40550
zipcode              0
beds             91254
state                0
stories         150673
mls-id          352193
PrivatePool     336825
MlsId            66869
target            2480
dtype: int64

In [106]:
# пропуски в признаках в процентном соотношении
for col_name, missings_count in missings.items():
    if missings_count > 0:
        percents = (missings_count / len(df)) * 100
        print(col_name, round(percents,2),'%')
        

status 10.58 %
private pool 98.89 %
propertyType 9.21 %
street 0.0 %
baths 28.19 %
fireplace 72.66 %
city 0.01 %
sqft 10.75 %
beds 24.2 %
stories 39.95 %
mls-id 93.39 %
PrivatePool 89.31 %
MlsId 17.73 %
target 0.66 %


### Признаки private pool и PrivatePool

Признаки **private pool** и **PrivatePool** содержат более 80% пропусков.

Согласно брифу Признаки **private pool** и **PrivatePool** содержат одинаковую информацию: наличие или отсутствие собственного бассейна. Вероятно, один содержит данные отсутсвующие в другом, а второй данные отсутсвующие в первом. Посмотрим на их содержание:

In [107]:
print('Уникальные значения в параметре private pool:', df['private pool'].unique())
print('Уникальные значения в параметре PrivatePool:', df['PrivatePool'].unique())

Уникальные значения в параметре private pool: [nan 'Yes']
Уникальные значения в параметре PrivatePool: [nan 'yes' 'Yes']


Признак **PrivatePool** содержит три варианта значения, в то время как **private pool** только два. Приведем к единообразию и объединим признаки:

In [108]:
df['PrivatePool'] = df['PrivatePool'].replace('yes', 'Yes')
#объединим два параметра используя метод DataFrame.combine_first, чтобы заполнить пропуски одного значением другого
df['PrivatePool'] = df['private pool'].combine_first(df['PrivatePool'])
# удалим столбец private pool
df = df.drop('private pool', axis=1)
#посмотрим на кол-во пропусков PrivatePool после объединения
print(df['PrivatePool'].isnull().sum())

332644


Пропуски в признаке **PrivatePool** указывают на отсутсвие собственного басейна, булево закодируем признаки, пусть наличие басейна будет *True*, а отсутсвие - *False*:

In [109]:
#заменим NaN значение на False
df['PrivatePool'] = df['PrivatePool'].fillna(False)
#заменим значение Yes на True
df['PrivatePool'] = df['PrivatePool'].replace('Yes', True)

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377135 entries, 0 to 377134
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337218 non-null  object
 1   propertyType  342402 non-null  object
 2   street        377133 non-null  object
 3   baths         270827 non-null  object
 4   homeFacts     377135 non-null  object
 5   fireplace     103112 non-null  object
 6   city          377101 non-null  object
 7   schools       377135 non-null  object
 8   sqft          336585 non-null  object
 9   zipcode       377135 non-null  object
 10  beds          285881 non-null  object
 11  state         377135 non-null  object
 12  stories       226462 non-null  object
 13  mls-id        24942 non-null   object
 14  PrivatePool   377135 non-null  bool  
 15  MlsId         310266 non-null  object
 16  target        374655 non-null  object
dtypes: bool(1), object(16)
memory usage: 46.4+ MB


Посмотрим на дубликаты после преобразования признака **PrivatePool**:

In [111]:
df.duplicated().sum()

0

### Признак fireplace

In [112]:
#посмотрим на общее кол-во уникальных значений признака
df['fireplace'].str.lower().nunique()

1652

In [113]:
#посмотрим на распределение уникальных значений признака
df['fireplace'].str.lower().value_counts().head(30)

yes                          71209
1                            14544
2                             2432
not applicable                1993
fireplace                      847
3                              564
living room                    433
location                       399
wood burning                   311
gas/gas logs                   300
no                             289
fireplace yn                   287
special features               279
1 fireplace                    274
0                              271
familyrm                       246
fireplace features             239
great room                     207
wood                           206
gas logs, great room           188
ceiling fan                    186
living room, wood burning      185
4                              184
family room, wood burning      149
familyrm, gas logs             146
gas                            134
gas logs, in great room        112
gas logs                       107
gas logs, in living 

Признак **fireplace** содержит множество различных вариаций, однако приобладающим значением является '*yes*'. По сути, его значимость заключается в определении наличия или отсутсвия камина. Давайте его бинарно закодируем. Наличие камина будет True, отсутсвие - False:

In [114]:
df['fireplace'] = df['fireplace'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none', '0', 'not applicable', 'no'] and not pd.isna(x) else False)

In [115]:
# посмотрим на соотношение вариаций признака после кодирования
df['fireplace'].value_counts()


False    274295
True     102840
Name: fireplace, dtype: int64

### Признак propertyType

Посмотрим на количество уникальных значений в **propertyType**:

In [116]:
df['propertyType'].nunique()

1280

Посмотрим на распределение уникальных значений:

In [117]:
df['propertyType'].value_counts().head(40)

single-family home               92199
Single Family                    62867
Single Family Home               31728
condo                            25964
lot/land                         20526
Condo                            16561
townhouse                        11456
Land                             10934
multi-family                      7900
Condo/Townhome/Row Home/Co-Op     7701
Townhouse                         6936
Traditional                       5913
coop                              3265
Multi Family                      2793
High Rise                         1823
Ranch                             1781
mobile/manufactured               1618
Detached, One Story               1614
Single Detached, Traditional      1581
Contemporary                      1557
Multi-Family Home                 1501
1 Story                           1234
Colonial                          1205
Mobile / Manufactured             1066
Contemporary/Modern               1000
2 Stories                

Видим, что многие значения различаются по форме написания, но суть есть одно. Преобразуем их, а также сформируем значение *Other* из значений не описывающих тип недвижимости:

In [118]:
#преобразуем все значения к строчному написанию
df.propertyType = df.propertyType.str.lower()
#приведем написание к единой форме,  так же заменим yes и unknown на Other
df.propertyType = df.propertyType \
.str.replace('single-family home','single family') \
.str.replace('single family home','single family') \
.str.replace('yes','Other') \
.str.replace('unknown','Other')
    
#выведем количество уникальных значений после преобразования
print('Количество уникальных значений после преобразования:{}'.format(df['propertyType'].nunique()))


Количество уникальных значений после преобразования:1269


В признаке много значений выполненных в форме перечисления через запятую, так же большое количество синонимов. Приобразуем эти данные:

In [119]:
#избавимся от избыточной информации
df['propertyType'] = df['propertyType'].str.split(',').str[0]
#функция преобразования синонимов
def syn_rename(value):
    for key, syn_val in synonyms.items():
        if value in syn_val:
            return key
    return 'other'

#словарь синонимов
synonyms = {
    'single_family_home': [
        'single family', '1 story', '2 story', 'detached', '1 story/ranch', '1 story traditional', 'single detached', 'singlefamilyresidence', 'single wide', 'single-wide mobile with land',
        'two story', 'one story', 'one level unit', 'rancher', '1 1/2 story', 'single wide mh', 'one story traditional'],
    'multi_family_home': [
        'multi-family', 'multi-family home', 'duplex', 'triplex', 'fourplex', 'multi_level', 'multi family', '2 unit condo', '2-story'],
    'condo': [
        'condo', 'coop', 'cooperative', 'condo/townhome/row home/co-op', 'condo/townhome', 'condominium', 'condo/unit', 'apartment/condo/townhouse', 'co-op', '2 story condo', 'high rise'],
    'townhouse': [
        'townhouse', 'townhome style', 'townhouse-interior', 'townhouse-end unit'],
    'apartment': [
        'apartment', 'condominium (single level)', 'high-rise', 'mid-rise', 'low-rise (1-3 stories)', 'Flats', 'studio'],
    'land': [
        'lot/land', 'land'],
    'mobile_home': [
        'mobile/manufactured', 'mobile / manufactured', 'manufactured house', 'mfd/mobile home', 'mobile home', 'manufactured home', 'manufactured double-wide', 'manufactured single-wide', 'mobile home 1 story', 'mobile manu - double wide'],
    'miscellaneous': [
        'miscellaneous'],
    'ranch': [
        'ranch', 'one story'],
    'modern': [
        'contemporary', 'contemporary/modern', "modern", 'mid-century modern', 'modern farmhouse', 'modernist'],
    'historical': [
        'historical', 'designated historical home', 'historical/conservation district', 'historic/older', 'historic vintage', 'historic'],
    'other': [
        'Other', 'english', 'urban contemporary', 'other style', 'florida', 'farms/ranches', 'carriage house', 'country english', 'straight thru', 'less than 4 floors', 'bungalow', 'custom', 'arts & crafts', 'tudor', 'new build 2019', 'split foyer', 'cottage', 'cottage/camp', 'garden home', 'farm/ranch', 'farm/ranch house', 'farm house', 'hi ranch', 'attached duplex', 'farmhouse', 'houseboat', 'ground floor', 'victorian', '3 story', '3+ stories', 'santa barbara/tuscan', 'old style', 'modular/prefab', 'post and beam', 'manuf/mobile', 'multiple occupancy', 'attached', 'hawaiian plantation', 'forest garden home', '1 1/2 story with basement', 'split-entry', 'texas hill country', 'lake house', '1 story with basement', 'hi-rise', 'coastal beach home', 'historical', 'key west/coastal', 'loft/balcony', 'english manor', 'mid-rise (4-7 stories)', 'mid-level', 'new englander', 'residential (<1 acre)', 'ranch','residential (1+ acre)', 'split', 'split level', 'split (4 level)', 'split (5+ level)', 'urban', 'patio', 'patio home', 'penthouse', 'manor', 'victorian/federal', 'coastal', 'coastal contemporary', 'coastal ii', 'coastal modern', 'coastal two story', 'mountain contemporary', 'key west', 'high ranch', 'end unit']
}

#преобразуем синонимы в нашем признаке
df['propertyType']= df['propertyType'].apply(syn_rename)

#смотрим количество пропусков и уникальных значений после приобразования
print(f'количество пропусков:{df.propertyType.isna().sum()}\n'
      f'количество уникальных значений:{df.propertyType.nunique()}\n'
      f'уникальные значения:\n{df.propertyType.value_counts()}')

количество пропусков:0
количество уникальных значений:12
уникальные значения:
single_family_home    195937
condo                  56958
other                  51584
land                   31460
townhouse              18543
multi_family_home      12349
mobile_home             3551
modern                  3252
ranch                   2033
apartment               1308
miscellaneous             96
historical                64
Name: propertyType, dtype: int64


### Признак stories

Признак **stories** содержит информацию об количестве этажей. Посмотрим на признак:

In [120]:
print(f"Количество уникальных значений:{df['stories'].nunique()}")
print(f"количество пропусков:{df['stories'].isna().sum()}")
print(f"Количество пропусков в процентном отношении:{round(df['stories'].isna().sum()/len(df)*100,2)}%")
print(f"распределение значений:\n{df['stories'].value_counts()}")

Количество уникальных значений:348
количество пропусков:150673
Количество пропусков в процентном отношении:39.95%
распределение значений:
1.0                                  67451
2.0                                  55283
1                                    23085
2                                    18146
3.0                                  11272
                                     ...  
Manufactured Home, Non-Site Built        1
Bedroom - Split Plan                     1
78                                       1
One, Two, Multi/Split                    1
65.0                                     1
Name: stories, Length: 348, dtype: int64


Посмотрим на распределение пропусков по типам объектов:

In [121]:
df[df.stories.isna()].propertyType.value_counts()

single_family_home    46341
other                 37042
land                  28883
condo                 24360
multi_family_home      5212
townhouse              3939
mobile_home            1499
ranch                  1389
modern                 1129
apartment               824
historical               44
miscellaneous            11
Name: propertyType, dtype: int64

Значение признака **stories** *single_family_home* наиболее представленно в данных, и может быть представлен как одноэтажными так и иными по этажности домами, посмотрим на отношение пропущенных данных по этому значению:

In [122]:
selected_sfh = df[(df.stories.isna()) & (df['propertyType'] == 'single_family_home')]
percent_sfh = len(selected_sfh)/len(df)*100
single_family_home = df[(df['propertyType'] == 'single_family_home')]
percent_single_family_home = len(selected_sfh)/len(single_family_home)*100

print(f"Количество пропусков в признаке stories по типу single_family_home в отношении ко всему объему данных: {round(percent_sfh,2)}%")
print(f"Количество пропусков в признаке stories по типу single_family_home в отношении к всему объему single_family_home: {round(percent_single_family_home,2)}%")

Количество пропусков в признаке stories по типу single_family_home в отношении ко всему объему данных: 12.29%
Количество пропусков в признаке stories по типу single_family_home в отношении к всему объему single_family_home: 23.65%


В признаке 39.95% пропусков распределенные по всем типам объектов, так же присутствует некоторое количество текстовых значений. И если в случае с land, condo, apartment, mobile_home, ranch мы можем принять условность, что этажность может быть ровна 0, то с остальными признаками - нет. Поиск и заполнение данных по каждому объекту не целесообразно, лучше удалить этот признак:

In [123]:
df = df.drop('stories', axis=1)

### Признак street

Признак **street** содержит информацию об адресе объекта. Посмотрим нуждается ли он в оптимизации:

In [124]:
df['street'].value_counts()

Address Not Disclosed        672
Undisclosed Address          517
(undisclosed Address)        391
Address Not Available        175
Unknown Address               72
                            ... 
1737 Parkview Green Cir        1
14890 Rockridge Ln             1
497 Kingswood Dr               1
202 Seacrest Beach Blvd W      1
5983 Midcrown Dr               1
Name: street, Length: 337076, dtype: int64

In [125]:
df['street'].isna().sum()

2

Мы имеем два пропуска и множестно значений обозначающих, что адрес не разглашается, либо недоступен, либо неизвестен. Примим идею, что все эти значения есть суть одно, объединим их, отнесем к ним пропуски:

In [126]:
#список синонимов
disclosed_adress = {
    'Undisclosed Address':['Address Not Disclosed', 'Undisclosed Address','(undisclosed Address)','Address Not Available', 'Unknown Address']
}
#функция замены синонимов
def undisclosed_Address(value):
    for key, val in disclosed_adress.items():
        if value in val:
            return key
    return value
#заменяем синонимы
df['street'] = df['street'].apply(undisclosed_Address)    
#заменяем пропуски на Undisclosed Address        
df['street'] = df['street'].fillna('Undisclosed Address')        

df['street'].value_counts()

Undisclosed Address          1829
2103 E State Hwy 21            57
11305 Gulf Fwy                 54
17030 Youngblood Rd.           38
NE 58th Cir                    34
                             ... 
1737 Parkview Green Cir         1
14890 Rockridge Ln              1
497 Kingswood Dr                1
202 Seacrest Beach Blvd W       1
5983 Midcrown Dr                1
Name: street, Length: 337072, dtype: int64

In [127]:
df['street'].nunique()

337072

Оптимизируем варианты признака приведя к нижнему регистру:

In [128]:
df['street'] = df['street'].str.lower()

In [129]:
df['street'].nunique()

336262

### Признак city

Есть пропуски попробуем востановить по признаку **zipcode** при помощи **geopy**:

In [130]:
#df_zip = df.copy()

In [131]:
#user_agent = UserAgent().random
#geolocator = Nominatim(user_agent=user_agent, timeout=10)

#def get_coordinates(address, retry_count=3):
#    if retry_count <= 0:
#       return None

#    try:
#        location = geolocator.geocode(address)
#        if location:
#            return (location.address)
#        else:
#            return None
#    except (GeocoderTimedOut, GeocoderUnavailable, GeocoderServiceError):
#        return get_coordinates(address, retry_count - 1)

#df_zip['location'] = df_zip['zipcode'].apply(get_coordinates)

In [132]:
#df_zip.head(5)

In [133]:
#df_zip[df_zip['city'].isna()]

In [134]:
#df_zip['city'].isna().sum()

Видим, что попытка решить через **geopy** не увенчалась успехом. Пропусков всего 34, заполним их в ручном режиме проверяя zip-code через https://tools.usps.com/  сверяя результат с кодом штата признака **state**:

In [135]:
df.at[7830,'city'] = 'Reddick'
df.at[21529,'city'] = 'MORRISTON'
df.at[26466,'city'] = 'LAREDO'
df.at[60990,'city'] = 'OCALA'
df.at[81509,'city'] = 'DUNNELLON'
df.at[122337,'city'] = 'OCALA'
df.at[123481,'city'] = 'PUNTA GORDA'
df.at[131266,'city'] = 'PORT CHARLOTTE'
df.at[151954,'city'] = 'OCALA'
df.at[170335,'city'] = 'KISSIMMEE'
df.at[171290,'city'] = 'LAREDO'
df.at[186779,'city'] = 'OCALA'
df.at[199648,'city'] = 'OCALA'
df.at[215069,'city'] = 'SILVER SPRINGS'
df.at[221264,'city'] = 'LAREDO'
df.at[221853,'city'] = 'LAREDO'
df.at[222785,'city'] = 'OCALA'
df.at[262327,'city'] = 'CLEVELAND'
df.at[270897,'city'] = 'LAREDO'
df.at[277436,'city'] = 'REDDICK'
df.at[278490,'city'] = 'KISSIMMEE'
df.at[281993,'city'] = 'KISSIMMEE'
df.at[304900,'city'] = 'KISSIMMEE'
df.at[309230,'city'] = 'KISSIMMEE'
df.at[312934,'city'] = 'DUNNELLON'
df.at[325746,'city'] = 'PUNTA GORDA'
df.at[331144,'city'] = 'OCKLAWAHA'
df.at[342308,'city'] = 'LAREDO'
df.at[346252,'city'] = 'MIAMI'
df.at[355353,'city'] = 'PUNTA GORDA'
df.at[358892,'city'] = 'KISSIMMEE'
df.at[370033,'city'] = 'HOUSTON'
df.at[370460,'city'] = 'OCKLAWAHA'

In [136]:
df[df['city'].isna()]

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,mls-id,PrivatePool,MlsId,target
130554,For sale,single_family_home,20003 mar rojo y mar egeo,Bathrooms: 6,"{'atAGlanceFacts': [{'factValue': '2000', 'fac...",False,NaN,"[{'rating': ['5/10', 'NA', '4/10'], 'data': {'...","Total interior livable area: 5,905 sqft",20003,5 bd,MA,A10761803,True,NaN,"$1,650,000"


Один объект имеет несоответсвие индекса и штата, также в признаке **street** указано что-то странное. Удалим его:

In [137]:
df = df.drop([130554])
print(f'колличество пропусков в признаке city:{df.city.isna().sum()}')

колличество пропусков в признаке city:0


Приведем признак к нижнему регистру:

In [138]:
df['city'] = df['city'].str.lower()

### Признак Status

Признак **Status** содержит информацию о статусе продажи объекта. Посмотрим на количество уникальных значений, пропусков:

In [139]:
print(f'количество пропусков:{df.status.isna().sum()}\n'
      f'количество уникальных значений:{df.status.nunique()}\n'
      f'распределение значений признака:\n{df.status.value_counts()}')

количество пропусков:39917
количество уникальных значений:159
распределение значений признака:
for sale                156058
Active                  105207
For sale                 43463
foreclosure               6425
New construction          5474
                         ...  
Contingent   No Show         1
Coming soon: Oct 24.         1
Coming soon: Oct 21.         1
Coming soon: Nov 14.         1
Coming soon: Dec 23.         1
Name: status, Length: 159, dtype: int64


In [140]:
print(f"уникальные значения:{df['status'].sort_values().unique()}\n")

уникальные значения:[' / auction' 'A Active' 'Accepted Offer' 'Accepting backups' 'Active'
 'Active - Auction' 'Active - Contingent' 'Active Backup'
 'Active Contingency' 'Active Contingent' 'Active Option'
 'Active Option Contract' 'Active Under Contract'
 'Active With Contingencies' 'Active With Offer' 'Active with Contract'
 'Active/Contingent' 'Apartment for rent' 'Auction' 'Auction - Active'
 'Back On Market' 'Back on Market' 'Backup' 'Backup Contract' 'C'
 'C Continue Show' 'CT Insp - Inspection Contingency' 'Closed'
 'Coming soon: Dec 1.' 'Coming soon: Dec 10.' 'Coming soon: Dec 11.'
 'Coming soon: Dec 12.' 'Coming soon: Dec 13.' 'Coming soon: Dec 14.'
 'Coming soon: Dec 15.' 'Coming soon: Dec 16.' 'Coming soon: Dec 18.'
 'Coming soon: Dec 2.' 'Coming soon: Dec 20.' 'Coming soon: Dec 23.'
 'Coming soon: Dec 24.' 'Coming soon: Dec 25.' 'Coming soon: Dec 27.'
 'Coming soon: Dec 3.' 'Coming soon: Dec 4.' 'Coming soon: Dec 5.'
 'Coming soon: Dec 6.' 'Coming soon: Dec 7.' 'Coming soo

Видим, что некоторые значения дублируются (различаются сокращением, регистром), так же присутствует признак *Coming soon*. Пропуски заменим на *No info*. Выполним необходимые преобразования:

In [141]:
#нормализуем регистра
df['status'] = df['status'].str.lower()
#словарь синонимов
stasus_synonims = {
    'pending':['option pending' ,  'pending (do not show)', 'pending ab',  'pending fe', 'pending offer approval',  'pending sh', 'pending w/escape clause', 'p', 'pending'],
    
    'pending Continue To Show':['pending   continue to show', 'pending - continue to show', 'pending continue to show', 'pending, continue to show', 'pending   continue to show   financing'],
      
    'Pending With Contingencies':['pending with contingencies','pending w/ cont.'],
    
    'Pending Inspection':['pending inspection', 'pending w/insp finance', 'pending in', 'pi'],
    
    'Pending Sale':['pending sale', 'p pending sale'],
    
    'Pending Taking Backups':['pending taking backups', 'pending - paking backups', 'pending bring backup', 'pending - backup offer requested','pending backups wanted', 'pending take backups', 'pending w/backup wanted'],
    
    'auction':[' / auction', 'a active', 'active', 'active - auction', 'auction - active', 'auction'],
    
    'closed':['c', 'closed'],  
    
    'Coming soon: Dec':['coming soon: dec 1.', 'coming soon: dec 10.', 'coming soon: dec 11.', 'coming soon: dec 12.', 'coming soon: dec 13.', 'coming soon: dec 14.',
                        'coming soon: dec 15.', 'coming soon: dec 16.', 'coming soon: dec 18.', 'coming soon: dec 2.', 'coming soon: dec 20.', 'coming soon: dec 23.',
                        'coming soon: dec 24.', 'coming soon: dec 25.', 'coming soon: dec 27.', 'coming soon: dec 3.', 'coming soon: dec 4.', 'coming soon: dec 5.',
                        'Coming soon: Dec 6.', 'Coming soon: Dec 7.', 'Coming soon: Dec 9.'],
    
    'Coming soon: Nov':['coming soon: nov 11.', 'coming soon: nov 12.', 'coming soon: nov 13.', 'coming soon: nov 14.', 'coming soon: nov 17.', 'coming soon: nov 19.',
                        'coming soon: nov 21.', 'coming soon: nov 22.', 'coming soon: nov 23.', 'coming soon: nov 25.', 'coming soon: nov 26.', 'coming soon: nov 27.',
                        'coming soon: nov 28.', 'coming soon: nov 29.', 'coming soon: nov 30.', 'coming soon: nov 5.', 'coming soon: nov 8.'],
    
    'Coming soon: Oct':['coming soon: oct 21.', 'coming soon: oct 24.', 'coming soon: oct 29.', 'coming soon: oct 30.'], 
    
    'Under Contract':['under contract', 'under contract'],
    
    'Under Contract Backups':['under contract backups', 'under contract taking back up offers', 'under contract w/ bckp'],
    
    'Under Contract Showing':['under contract   showing', 'uc continue to show', 'under contract - show',  'under contract show'],
    
    'Pre-foreclosure':[ 'pre-foreclosure', 'pf']
    
    }
#функция замены синонимов
def status_rename(value):
  for key, val in stasus_synonims.items():
    if value in val:
      return key
  return value
#заменяем синонимы
df['status'] = df['status'].apply(status_rename)
#заменяем пропуски на No info
df['status'] = df['status'].fillna('No info')

Смотрим на результаты после преобразования:

In [142]:
print(f'количество пропусков:{df.status.isna().sum()}\n'
      f'количество уникальных значений:{df.status.nunique()}\n'
      f'распределение значений признака:\n{df.status.value_counts()}')

количество пропусков:0
количество уникальных значений:83
распределение значений признака:
for sale                 199523
auction                  107160
No info                   39917
foreclosure                6768
pending                    6423
                          ...  
conting accpt backups         1
contingent   no show          1
conditional contract          1
contract                      1
backup                        1
Name: status, Length: 83, dtype: int64


### Признак baths

Признак **baths** демонстрирует количество ванных комнат. Посмотрим на количество уникальных значений и пропусков:

In [143]:
print(f'количество пропусков:{df.baths.isna().sum()}\n'
      f'количество уникальных значений:{df.baths.nunique()}\n'
      f'распределение значений признака:\n{df.baths.value_counts()}')

количество пропусков:106308
количество уникальных значений:229
распределение значений признака:
2 Baths       52458
3 Baths       35503
2             20452
2.0           16575
4 Baths       14763
              ...  
4.75 Baths        1
5.25 Baths        1
41.0              1
1.8 Baths         1
44.0              1
Name: baths, Length: 229, dtype: int64


In [144]:
df[df.baths.isna()].propertyType.value_counts()

other                 39846
land                  27147
single_family_home    19121
condo                 11433
multi_family_home      2316
modern                 2278
townhouse              1556
ranch                  1518
apartment               701
mobile_home             353
historical               32
miscellaneous             7
Name: propertyType, dtype: int64

Посмотрим на значения представленные в признаке, интерпретируем нетепичные значения:

In [145]:
l=list(df['baths'].unique())
#l
selected_row = df[(df['baths'] == '1 / 1-0 / 1-0 / 1-0') | (df['baths'] == '2-1 / 2-1 / 1-1 / 1-1') | (df['baths'] == '1 / 1 / 1 / 1') | (df['baths'] == '3-1 / 2-2')\
    | (df['baths'] == '116 / 116 / 116') | (df['baths'] == '1-0 / 1-0 / 1')] 

display(selected_row)

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,mls-id,PrivatePool,MlsId,target
24724,auction,multi_family_home,1811 beechwood ave,1-0 / 1-0 / 1,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",False,nashville,"[{'rating': ['6', '5', '3', '4', '5', 'NR', 'N...",2932,37212,3.0,TN,NaN,False,2081563,"$775,000"
63510,auction,multi_family_home,321 larkin springs rd,116 / 116 / 116,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",False,madison,"[{'rating': ['1', 'NR', '3', '1', '2', '1', 'N...",99999,37115,NaN,TN,NaN,False,2051320,"$1,600,000"
85354,auction,multi_family_home,6511 robertson ave,3-1 / 2-2,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",False,nashville,"[{'rating': ['2', '4', '3', '3', '2', 'NR', 'N...",1939,37209,3.0,TN,NaN,False,2085088,"$5,750,000"
110090,auction,multi_family_home,3317 ezell rd,1 / 1 / 1 / 1,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",False,nashville,"[{'rating': ['2', '3', '2', '1', '2', '2', 'NR...",4408,37211,3.0,TN,NaN,False,2027355,"$985,000"
301145,auction,multi_family_home,214 3rd ave,2-1 / 2-1 / 1-1 / 1-1,"{'atAGlanceFacts': [{'factValue': '1900', 'fac...",False,nashville,"[{'rating': ['1', '2', '3', '1', '4', '10', 'N...",11740,37201,2.0,TN,NaN,False,2107281,"$7,450,000"
316452,auction,multi_family_home,126 welworth st,1 / 1-0 / 1-0 / 1-0,"{'atAGlanceFacts': [{'factValue': '1966', 'fac...",False,madison,"[{'rating': ['3', '2', '3', '1', '2', '1', 'NR...",1324,37115,2.0,TN,NaN,False,2094893,"$2,250,000"


Видим, что через */* указывается этаж и количество ванных на этаж, в одном объекте указано *116 / 116 / 116* (63510). Объект 63510 оказался землей под строительство многоквартирного дома, возможно 116 это число предполагаемых квартир, может нет. Площадь этого объекта так же указана случайным набором цивф *99999*, лучше его удалить.

Видим, что в значениях много буквенных литералов, распределение пропусков сконцентрированно в типе имущества *other, land*. Уберем буквенные значения, заполним пропуски в *other, land, ranch, mobile_home* на 0, вместо прочерков поставим 0, в остальных значения проставим соответствующее значение, объект под индексом 63510 - удалим, переведем данные признака в тип *float*: 

In [146]:
#убираем буквенные литералы
df['baths'] = df['baths'].str.replace('[a-zA-Z+:]','', regex=True)

#заменяем значения 
df['baths'].replace(['1 / 1 / 1 / 1','2-1 / 2-1 / 1-1 / 1-1'],4,inplace=True)
df['baths'].replace(['1-2 ','1-0 / 1-0 / 1','1 / 1-0 / 1-0 / 1-0'],1,inplace=True)
df['baths'].replace('3-1 / 2-2',3,inplace=True)
df['baths'].replace(['~','0 / 0','. . ','-- ','—',' ',''],0,inplace=True)
df['baths'].replace(['7,500'], 7.5, inplace=True)
df['baths'].replace(['5,000'], 5, inplace=True)
df['baths'].replace(['3,500'], 3.5, inplace=True)
df['baths'].replace(['2,750'], 2.75, inplace=True)
df['baths'].replace(['2,250'], 2.25, inplace=True)
df['baths'].replace(['1,250'], 1.25, inplace=True)
df['baths'].replace(['1,500'], 1.5, inplace=True)
df['baths'].replace(['4,000'], 4, inplace=True)
df['baths'].replace(['2,000'], 2, inplace=True)
df['baths'].replace(['3,000'], 3, inplace=True)
df['baths'].replace(['2,500'], 2.5, inplace=True)
df['baths'].replace(['1,000'], 1, inplace=True)
df['baths'].replace(['1,750'], 1.75, inplace=True)


#удаляем объект по индексу 63510
df['baths'] = df['baths'].drop([63510])
#заменяем NaN на 0
df['baths'] = df['baths'].fillna(0)

#переведем в тип float и округлим до целого
df['baths'] = df['baths'].astype(float).round()



Посмотрим на распределение нулевых значений:

In [147]:

selected_lands = df[(df['baths'] == 0) & ((df['propertyType'] == 'other') | (df['propertyType'] == 'land') | (df['propertyType'] == 'ranch'))]

percent_lands = len(selected_lands) / len(df) * 100

selected_no_lands = df[(df['baths'] == 0) & ~(df['propertyType'].isin(['other', 'land', 'ranch']))]

percent_no_lands = len(selected_no_lands) / len(df) * 100

print( f"Количество уникальных значений: {df.baths.nunique()}\n")
print(f"Количество нулевых значений во всех объектах:{percent_lands+percent_no_lands:.2f}%")
print(f"Процент нулевых значений в объектах типа other, land, ranch: {percent_lands:.2f}%")
print(f"Процент нулиеых значений в объектах не относящихся к типам other, land, ranch: {percent_no_lands:.2f}%")

Количество уникальных значений: 48

Количество нулевых значений во всех объектах:29.62%
Процент нулевых значений в объектах типа other, land, ranch: 19.37%
Процент нулиеых значений в объектах не относящихся к типам other, land, ranch: 10.25%


Таким образом у нас 29.62% объектов не имеют ванных комнат. Из их 19.37% объекты относящиеся к типу *другое* либо одному из типов земель, что представляется логиным. В остальных типах 10.25% объектов без ванной комнаты, что возможно.

### Признак sqft

Признак **sqft** содержит информацию о площади объекта.Посмотрим на признак:

In [148]:
print(f"Количество уникальных значений:{df['sqft'].nunique()}")
print(f"количество пропусков:{df['sqft'].isna().sum()}")
print(f"Количество пропусков в процентном отношении:{round(df['sqft'].isna().sum()/len(df)*100,2)}%")
print(f"распределение значений:\n{df['sqft'].value_counts()}")

Количество уникальных значений:25404
количество пропусков:40550
Количество пропусков в процентном отношении:10.75%
распределение значений:
0                                          11854
1,200 sqft                                   839
1,000 sqft                                   654
1,100 sqft                                   573
1,800 sqft                                   563
                                           ...  
11,491                                         1
9,914                                          1
Total interior livable area: 3,055 sqft        1
5,177                                          1
Total interior livable area: 4,615 sqft        1
Name: sqft, Length: 25404, dtype: int64


В признаке присутствуют буквенные значения и пропуски, уберем их, переведем признак в int:

In [149]:
#выбираем все значения не nan
df_sqft = df.sqft[df.sqft.notna()]
#выбираем все "числовые" значения
df_sqft = df_sqft.str.findall(r"\d+")
#преобразуем 
df_sqft = df_sqft.apply(lambda x: ''.join(x))
#заполняем значения признака sqft получившимися значениями
df.loc[df.sqft.notna(), 'sqft']=df_sqft
#меняем пропуски на 0
df['sqft'] = df['sqft'].fillna(0)
#заменим пустые строки на 0
df.loc[df.sqft=='','sqft'] = 0
#приобразуем в тим int
df.sqft = df.sqft.astype(int)

In [150]:
print(f"Количество уникальных значений:{df['sqft'].nunique()}")
print(f"количество пропусков:{df['sqft'].isna().sum()}")
print(f"Количество пропусков в процентном отношении:{round(df['sqft'].isna().sum()/len(df)*100,2)}%")
print(f"распределение значений:\n{df['sqft'].value_counts()}")

Количество уникальных значений:9897
количество пропусков:0
Количество пропусков в процентном отношении:0.0%
распределение значений:
0          53224
1200        1426
1000        1027
1500        1014
1800         988
           ...  
5728968        1
12503          1
8815           1
15391          1
13870          1
Name: sqft, Length: 9897, dtype: int64


Посмотрем на распределение нулей по типам объектов:

In [151]:
selected_sfh = df[(df['sqft'] == 0) & (df['propertyType'] == 'single_family_home')]
percent_sfh = len(selected_sfh)/len(df)*100
single_family_home = df[(df['propertyType'] == 'single_family_home')]
percent_single_family_home = len(selected_sfh)/len(single_family_home)*100

selected_condo = df[(df['sqft'] == 0) & (df['propertyType'] == 'condo')]
percent_c = len(selected_condo)/len(df)*100
condo = df[(df['propertyType'] == 'condo')]
percent_condo = len(selected_condo)/len(condo)*100

selected_other = df[(df['sqft'] == 0) & (df['propertyType'] == 'other')]
percent_oth = len(selected_other)/len(df)*100
other = df[(df['propertyType'] == 'other')]
percent_other = len(selected_other)/len(other)*100

selected_land = df[(df['sqft'] == 0) & (df['propertyType'] == 'land')]
percent_la = len(selected_land)/len(df)*100
land = df[(df['propertyType'] == 'land')]
percent_land = len(selected_land)/len(land)*100

selected_townhouse = df[(df['sqft'] == 0) & (df['propertyType'] == 'townhouse')]
percent_tow = len(selected_townhouse)/len(df)*100
townhouse = df[(df['propertyType'] == 'townhouse')]
percent_townhouse = len(selected_townhouse)/len(townhouse)*100

selected_multi_family_home = df[(df['sqft'] == 0) & (df['propertyType'] == 'multi_family_home')]
percent_multi = len(selected_multi_family_home)/len(df)*100
multi_family_home = df[(df['propertyType'] == 'multi_family_home')]
percent_multi_family_home = len(selected_multi_family_home)/len(multi_family_home)*100

selected_mobile_home = df[(df['sqft'] == 0) & (df['propertyType'] == 'mobile_home')]
percent_mobile = len(selected_mobile_home)/len(df)*100
mobile_home = df[(df['propertyType'] == 'mobile_home')]
percent_mobile_home = len(selected_mobile_home)/len(mobile_home)*100

selected_modern = df[(df['sqft'] == 0) & (df['propertyType'] == 'modern')]
percent_mod = len(selected_modern)/len(df)*100
modern = df[(df['propertyType'] == 'modern')]
percent_modern = len(selected_modern)/len(modern)*100

selected_ranch = df[(df['sqft'] == 0) & (df['propertyType'] == 'ranch')]
percent_ran = len(selected_ranch)/len(df)*100
ranch = df[(df['propertyType'] == 'ranch')]
percent_ranch = len(selected_ranch)/len(ranch)*100

selected_apartment = df[(df['sqft'] == 0) & (df['propertyType'] == 'apartment')]
percent_apart = len(selected_apartment)/len(df)*100
apartment = df[(df['propertyType'] == 'apartment')]
percent_apartment = len(selected_apartment)/len(apartment)*100

selected_miscellaneous = df[(df['sqft'] == 0) & (df['propertyType'] == 'miscellaneous')]
percent_misc = len(selected_miscellaneous)/len(df)*100
miscellaneous = df[(df['propertyType'] == 'miscellaneous')]
percent_miscellaneous = len(selected_miscellaneous)/len(miscellaneous)*100

selected_historical = df[(df['sqft'] == 0) & (df['propertyType'] == 'historical')]
percent_hist = len(selected_miscellaneous)/len(df)*100
historical = df[(df['propertyType'] == 'historical')]
percent_historical = len(selected_miscellaneous)/len(historical)*100

type_zero = round((len(df[(df['sqft']==0)])/len(df)*100),2)

d = {'scope':['in', 'all'], 'single_family_home':[(round(percent_single_family_home,2)),(round(percent_sfh,2))],
     'condo':[(round(percent_condo,2)),(round(percent_c,2))],
     'other':[(round(percent_other,2)),(round(percent_oth,2))],
     'land':[(round(percent_land,2)),(round(percent_la,2))],
     'townhouse':[(round(percent_townhouse,2)),(round(percent_tow,2))],
     'multi_family_home':[(round(percent_multi_family_home,2)),(round(percent_multi,2))],
     'mobile_home':[(round(percent_mobile_home,2)),(round(percent_mobile,2))],
     'modern':[(round(percent_modern,2)),(round(percent_mod,2))],
     'ranch':[(round(percent_ranch,2)),(round(percent_ran,2))],
     'apartment':[(round(percent_apartment,2)),(round(percent_apart,2))],
     'miscellaneous':[(round(percent_miscellaneous,2)),(round(percent_misc,2))],
     'historical':[(round(percent_historical,2)),(round(percent_hist,2))]}
scope = pd.DataFrame(data=d)

display(scope)
print(f"Количество нулевых значений признака sqft от общего числа строк:{type_zero}%")


,scope,single_family_home,condo,other,land,townhouse,multi_family_home,mobile_home,modern,ranch,apartment,miscellaneous,historical
0,in,1.74,3.56,36.53,86.94,1.90,7.22,6.42,0.98,2.02,3.59,4.17,6.25
1,all,0.90,0.54,5.00,7.25,0.09,0.24,0.06,0.01,0.01,0.01,0.00,0.00


Количество нулевых значений признака sqft от общего числа строк:14.11%


Значительная часть находится в значениях other,land (12.25% от 14.11% нулей в признаке). Значения other и land имеют большое количество нулевых значения, что скажется на качестве предсказания по этим типам объекта. В значениях обозначающих типы строений только 2.14% нулевых значений, так что пока оставим как есть.

### Признак beds

Признак **beds** указывает на количество спален в объекте. Посмотри на него:

In [152]:
print(f"Количество уникальных значений:{df['beds'].nunique()}")
print(f"количество пропусков:{df['beds'].isna().sum()}")
print(f"Количество пропусков в процентном отношении:{round(df['beds'].isna().sum()/len(df)*100,2)}%")
print(f"распределение значений:\n{df['beds'].value_counts()}")

Количество уникальных значений:1184
количество пропусков:91254
Количество пропусков в процентном отношении:24.2%
распределение значений:
3 Beds         53454
4 Beds         35412
3              31405
2 Beds         26355
4              20030
               ...  
8,023 sqft         1
10,193 sqft        1
8.93 acres         1
5,510 sqft         1
8,479 sqft         1
Name: beds, Length: 1184, dtype: int64


Посмотрим на распределение пропусков по типу объекта:

In [153]:
df[df.beds.isna()].propertyType.value_counts()

other                 36492
land                  28854
condo                 10739
single_family_home     8349
modern                 2265
multi_family_home      1648
ranch                  1508
apartment               623
townhouse               571
mobile_home             171
historical               34
Name: propertyType, dtype: int64

In [154]:
l = list(df['beds'].unique())
l

['4',
 '3 Beds',
 '5 Beds',
 nan,
 '2 Beds',
 '3',
 '4 Beds',
 '3 bd',
 'Baths',
 '4 bd',
 '5 bd',
 '8 Beds',
 '2',
 '2 bd',
 '2.0',
 '5',
 '1',
 '3.0',
 '6 Beds',
 'Bath',
 '12 Beds',
 '6',
 '14',
 '0.25 acres',
 '6 bd',
 '-- bd',
 '7',
 '5.0',
 '4.0',
 ' ',
 '7 Beds',
 '1 bd',
 '9 Beds',
 '0.44 acres',
 '0',
 '1.43 acres',
 '0.32 acres',
 '8,637 sqft',
 '7.0',
 '8',
 '16 Beds',
 '9.7 acres',
 '1.0',
 '16 bd',
 '10,310 sqft',
 '6,250 sqft',
 '3.02 acres',
 '2 acres',
 '5,000 sqft',
 '0.96 acres',
 '6.0',
 '13 bd',
 '6.31 acres',
 '0.6 acres',
 '9',
 '5,662 sqft',
 '2,200 sqft',
 '10',
 '18 Beds',
 '10 Beds',
 '0.46 acres',
 '0.59 acres',
 '20 Beds',
 '0.0',
 '2,874 sqft',
 '0.26 acres',
 '10 bd',
 '32',
 '0.4 acres',
 '2,178 sqft',
 '6,351 sqft',
 '4,356 sqft',
 '5 acres',
 '0.55 acres',
 '5,501 sqft',
 '8,001 sqft',
 '0.7 acres',
 '1.69 acres',
 '0.88 acres',
 '0.34 acres',
 '13 Beds',
 '1.5 acres',
 '0.97 acres',
 '7,405 sqft',
 '7 bd',
 '8.43 acres',
 '1,502 sqft',
 '8,712 sqft',
 

В признаке присутствует множестно данных отличных от количества кроватей, давайте посмотрим на количество значений с меткой *sqft* и *acres*, возможно ими можно заполнить пропуски в признаке **sqft**:

In [155]:
df['beds'].str.endswith('acres',na=False).value_counts()

False    375542
True       1592
Name: beds, dtype: int64

In [156]:
df['beds'].str.endswith('sqft',na=False).value_counts()

False    375699
True       1435
Name: beds, dtype: int64

Искомых значений оказалось незначительное количество. Обилие пропусков, мусорных данных делает признак **beds** бесполезным - удалим его:

In [157]:
df = df.drop('beds', axis=1)

### Признак homeFacts

Признак **homeFacts** содержит сведения о строительстве объекта. Посмотрим на него:

In [158]:
print(f"Количество уникальных значений:{df['homeFacts'].nunique()}")
print(f"количество пропусков:{df['homeFacts'].isna().sum()}")
print(f"Количество пропусков в процентном отношении:{round(df['homeFacts'].isna().sum()/len(df)*100,2)}%")
print(f"распределение значений:\n{df['homeFacts'].value_counts()}")

Количество уникальных значений:321008
количество пропусков:0
Количество пропусков в процентном отношении:0.0%
распределение значений:
{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': '—', 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}                                                       7174
{'atAGlanceFacts': [{'factValue': None, 'factLabel': 'Year built'}, {'factValue': None, 'factLabel': 'Remodeled year'}, {'factValue': None, 'factLabel': 'Heating'}, {'factValue': None, 'factLabel': 'Cooling'}, {'factValue': None, 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}                                            3532
{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'fact

Признак содержит две группы фактов об объекте *'factValue'* и *'factLabel'* объединенные в словарь. Изымим эти факты в отдельные признаки, посмотрим будут ли они нам полезны:

In [159]:
#С помощью регулярных выражений выдерним из признака значения для Value и Label
Value = df.homeFacts.str.findall(r"\bfactValue': ([\s\S]+?), 'factLabel\b")
label = df.homeFacts.str.findall(r"\bfactLabel': ([\s\S]+?)[}\b]")
#создадим список названий признаков, удалив лишние кавычки
list_label = ','.join(label[0]).replace("'","").split(',')
list_label

['Year built',
 'Remodeled year',
 'Heating',
 'Cooling',
 'Parking',
 'lotsize',
 'Price/sqft']

In [160]:
# Создадим эти признаки в нашем фрейме и заполним значениями
for i, val in enumerate(list_label):
    df[val]=Value.apply(lambda x: x[i])

Признак **homeFacts** более не нужен, удалим его:

In [161]:
df = df.drop('homeFacts', axis=1)

In [162]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 377134 entries, 0 to 377134
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   status          377134 non-null  object 
 1   propertyType    377134 non-null  object 
 2   street          377134 non-null  object 
 3   baths           377134 non-null  float64
 4   fireplace       377134 non-null  bool   
 5   city            377134 non-null  object 
 6   schools         377134 non-null  object 
 7   sqft            377134 non-null  int32  
 8   zipcode         377134 non-null  object 
 9   state           377134 non-null  object 
 10  mls-id          24941 non-null   object 
 11  PrivatePool     377134 non-null  bool   
 12  MlsId           310266 non-null  object 
 13  target          374654 non-null  object 
 14  Year built      377134 non-null  object 
 15  Remodeled year  377134 non-null  object 
 16  Heating         377134 non-null  object 
 17  Cooling   

#### Признак Year built

Признак **Year built** указывает на год строительства объекта, посмотрим на него:

In [163]:
df['Year built'].unique()[:10]

array(["'2019'", "'1961'", "'2006'", "''", "'1920'", "'1976'", "'1970'",
       "'1965'", "'2015'", "'1996'"], dtype=object)

Уберем ненужные ковычки:

In [164]:
df['Year built'] = df['Year built'].str.replace("'",'', regex=True)

Изучем значения признака:

In [165]:
l = list(df['Year built'].unique())
l

['2019',
 '1961',
 '2006',
 '',
 '1920',
 '1976',
 '1970',
 '1965',
 '2015',
 '1996',
 '1982',
 '1905',
 '2008',
 '1899',
 '2016',
 '1954',
 '1989',
 '2014',
 '1915',
 '1994',
 '1910',
 '1923',
 '1960',
 '1977',
 '1972',
 '1963',
 '1981',
 '1949',
 '1930',
 '1991',
 '1958',
 '2005',
 '2017',
 '2018',
 '1983',
 '1928',
 '1998',
 '1997',
 '1964',
 '1974',
 '1940',
 '1973',
 '1968',
 '1999',
 '1918',
 '2007',
 '1984',
 '1946',
 '2003',
 '2000',
 '2004',
 '1969',
 '2002',
 '1952',
 '1947',
 '1957',
 '1978',
 '1956',
 '1967',
 '1975',
 '1988',
 '1955',
 '1950',
 '1959',
 '2012',
 '1924',
 '1966',
 '1992',
 '2001',
 '1995',
 '1985',
 '2010',
 '1971',
 '1909',
 '1953',
 '1993',
 '1990',
 '1980',
 '1951',
 '1907',
 '1885',
 '1941',
 '1927',
 '1929',
 '1901',
 'None',
 '1922',
 '1906',
 '1945',
 '1979',
 '2009',
 '1962',
 '1913',
 '2011',
 '1897',
 '1987',
 '2020',
 '1926',
 '1939',
 '1925',
 '2013',
 '1986',
 'No Data',
 '1943',
 '1935',
 '1890',
 '1900',
 '1895',
 '1911',
 '1948',
 '1942',
 '

Есть дома со странными датами:

In [166]:
house_strange = df[(df['Year built'] == '1019') | (df['Year built'] == '1057') | (df['Year built'] == '1060')  | (df['Year built'] == '1') |
                   (df['Year built'] == '1208') | (df['Year built'] == '2025') | (df['Year built'] == '559990649990')]
house_strange

,status,propertyType,street,baths,fireplace,city,schools,sqft,zipcode,state,...,PrivatePool,MlsId,target,Year built,Remodeled year,Heating,Cooling,Parking,lotsize,Price/sqft
77797,auction,condo,12522 15th ave ne,2.0,False,seattle,"[{'rating': ['5', '6', '7', '6', '7', 'NR', 'N...",1486,98125,WA,...,False,161732709,"$649,990",559990649990,'','','','',None,'$437'
120334,for sale,land,w trinity ln,0.0,False,nashville,"[{'rating': ['2/10', '1/10', 'None/10'], 'data...",1,37207,TN,...,False,1997063,"$5,950,000",1,'1111','','','','12.59 acres',"'$5,950,000/sqft'"
149913,Pending Inspection,single_family_home,648 n mulberry st,2.0,False,logan,"[{'rating': ['9', '6', '4', '6', 'NR', 'NR'], ...",1208,43138,OH,...,False,9922581,"$79,000",1208,'','GAS FORCED AIR','','888',None,'$65'
164154,auction,other,2421 flume gate ct,0.0,True,raleigh,"[{'rating': ['5', '4', '3'], 'data': {'Distanc...",3518,27603,NC,...,False,2226990,"534,000",2025,'','','','','—','$152 / Sq. Ft.'
172710,auction,single_family_home,5532 nw 57th pl,2.0,False,ocala,"[{'rating': ['1', '4', '3', '1', '6', '5', 'NR...",1042,34482,FL,...,False,562474,"$104,900",1057,'','Central Electric','Central Electric','','8712','$101'
179733,for sale,land,5754 grande lagoon blvd,0.0,False,pensacola,"[{'rating': ['3/10', '7/10', '3/10'], 'data': ...",20900,32507,FL,...,False,501274,"$139,000",1,'','','','','0.48 acres','$7/sqft'
264030,auction,other,576 19th st,0.0,False,brooklyn,"[{'rating': ['6', '8', '5', '6'], 'data': {'Di...",1892,11218,NY,...,False,1131116,"1,499,999",1060,'1960','','','',"'1,953 Sq. Ft.'",'$793 / Sq. Ft.'
312838,auction,other,ellisway rd #13,3.0,False,charlotte,"[{'rating': [], 'data': {'Distance': [], 'Grad...",1811,28216,NC,...,False,3564047,"285,500",1019,'','',"'Heat Pump for A/C, Heat Pump for Heat'",'','—','$158 / Sq. Ft.'
313472,auction,other,2405 flume gate ct,0.0,True,raleigh,"[{'rating': ['5', '4', '3'], 'data': {'Distanc...",3000,27603,NC,...,False,2226994,"494,990",2025,'','','','','0.77 Acres','$165 / Sq. Ft.'
354101,auction,condo,12522 15th ave ne,2.0,False,seattle,"[{'rating': ['5', '6', '7', '6', '7', 'NR', 'N...",1233,98125,WA,...,False,161732708,"$559,990",559990649990,'','','','',None,'$454'


Домов со странными датами немного, их можно найти и внести вручную. Также есть *ковычки, None, No Data*, заменим их на *no info*:

In [167]:
df.at[77797,'Year built'] = '2021'
df.at[120334,'Year built'] = 'no info'
df.at[149913,'Year built'] = '1908'
df.at[164154,'Year built'] = '2021'
df.at[172710,'Year built'] = '1987'
df.at[179733,'Year built'] = 'no info'
df.at[264030,'Year built'] = '1960'
df.at[312838,'Year built'] = '2020'
df.at[313472,'Year built'] = '2020'
df.at[354101,'Year built'] = '2021'

df['Year built'] = df['Year built'].str.replace('^\s*$','no info', regex=True)

mask= df['Year built'].isin(['No Data','None'])
df.loc[mask,'Year built'] = 'no info'

Посмотрим на количество значений *no info* относительно датафрема в целом:

In [168]:
no_info_row = df[df['Year built'] == 'no info']

print(f"Количество значения 'no info' относительно датафрейма:{round(len(no_info_row)/len(df)*100,2)}%")

Количество значения 'no info' относительно датафрейма:16.86%


Допустим, данные о годе сроительства отсутствуют у типов объекта *other, land, ranch*. Посмотрим на количество значений *no info* относительно этих типа объекта:

In [169]:
no_info_row = df[(df['Year built'] == 'no info') & ((df['propertyType'] == 'other') | (df['propertyType'] == 'land') | (df['propertyType'] == 'ranch'))]

print(f"Количество значения 'no info' относительно other, land, ranch:{round(len(no_info_row)/len(df)*100,2)}%")

Количество значения 'no info' относительно other, land, ranch:11.89%


Таким образов видим, что данные о годе строительства отсутсвуют у 16.86% наблюдений, из них 11.89% это объекты other, land, ranch и 4.97% у других объектов, что является допустимым.

#### Признак Remodeled year

Признак **Remodeled year** указывает на год реконструкции объекта. Посмотрим на него:

In [170]:
#уберем ненужные ковычки
df['Remodeled year'] = df['Remodeled year'].str.replace("'",'', regex=True)
df['Remodeled year'].unique()[:10]

array(['', '1967', '2006', 'None', '2015', '1997', '2019', '1982', '1905',
       '2009'], dtype=object)

In [171]:
l = list(df['Remodeled year'].unique())
l

['',
 '1967',
 '2006',
 'None',
 '2015',
 '1997',
 '2019',
 '1982',
 '1905',
 '2009',
 '2007',
 '1996',
 '1995',
 '1988',
 '1972',
 '1949',
 '1954',
 '1930',
 '2003',
 '2000',
 '1994',
 '2005',
 '1983',
 '1965',
 '1968',
 '1993',
 '1999',
 '2014',
 '2008',
 '2010',
 '2004',
 '2002',
 '1984',
 '1952',
 '1978',
 '1975',
 '1971',
 '1947',
 '1957',
 '1950',
 '2012',
 '1973',
 '1970',
 '2016',
 '1989',
 '1998',
 '1980',
 '1986',
 '1955',
 '1991',
 '2011',
 '2017',
 '1941',
 '2001',
 '1990',
 '1979',
 '1901',
 '1918',
 '1977',
 '1962',
 '1956',
 '2013',
 '1960',
 '1981',
 '1969',
 '1987',
 '1958',
 '1910',
 '1925',
 '1964',
 '1966',
 '1992',
 '1943',
 '1961',
 '1948',
 '1931',
 '1976',
 '1974',
 '1935',
 '1940',
 '2018',
 '1916',
 '1914',
 '1963',
 '1985',
 '1951',
 '1944',
 '1942',
 '1959',
 '1927',
 '1946',
 '1945',
 '1920',
 '1938',
 '1936',
 '0',
 '1939',
 '1915',
 '1928',
 '1923',
 '1929',
 '1898',
 '1899',
 '1934',
 '1953',
 '1926',
 '1937',
 '1912',
 '1917',
 '1906',
 '1932',
 '1919',

Проверим, относится ли *None* приимущественно к other, land, ranch:

In [172]:
no_info_row = df[(df['Remodeled year'] == 'None') & ((df['propertyType'] == 'other') | (df['propertyType'] == 'land') | (df['propertyType'] == 'ranch'))]

print(f"Количество значения 'None' относительно other, land, ranch:{round(len(no_info_row)/len(df)*100,2)}%")

Количество значения 'None' относительно other, land, ranch:0.94%


Заменим некоректные значения на *None*, предполагая, что дом не реконструировался:

In [173]:
#количество значений None до преобразования
print(f"Процент None: {round(df[df['Remodeled year']=='None'].shape[0]/df.shape[0]*100, 2)}%")
print(f"Количество None: {df.loc[df['Remodeled year']=='None', 'Remodeled year'].count()}")

Процент None: 7.04%
Количество None: 26567


In [174]:
mask= df['Remodeled year'].isin(['', '0', '1111'])
df.loc[mask,'Remodeled year'] = 'None'

In [175]:
#количество значений None после преобразования
print(f"Процент None: {round(df[df['Remodeled year']=='None'].shape[0]/df.shape[0]*100, 2)}%")
print(f"Количество None: {df.loc[df['Remodeled year']=='None', 'Remodeled year'].count()}")

Процент None: 60.01%


Количество None: 226332


Предположим, что отсутсвие информации о реконструкции связанно с новизной объекта. Посмотрим на количество объектов старше 23 лет (старше 2000 года) без реконструкции, их отношение к общему количеству наблюдений: 

In [176]:
with_none = df[(df['Remodeled year'] == 'None') | df['Year built']]
print(f"Количество объектов построенных ранее 2000-го года и не реконструированнных:{with_none[with_none['Year built'] < '2000'].shape[0]}")
print(f"Отношение объектов построенных ранее 2000-го года и не реконструированнных к общему числу наблюдений:{round(len(with_none[with_none['Year built'] < '2000'])/len(df)*100,2)}%")

Количество объектов построенных ранее 2000-го года и не реконструированнных:195368
Отношение объектов построенных ранее 2000-го года и не реконструированнных к общему числу наблюдений:51.8%


Реконструкция несомненно влияет на стоимость объекта в сторону её увеличения, однако в наших данных более половины объектов не имеющих данных о реконструкции построенны ранее 2000-го. По всей видимости это не отсутсвия реконструкции, а отсутсвие данных о реконструкции, что делает признак бесполезным. Удалим его:

In [177]:
df = df.drop('Remodeled year', axis=1)

#### Признак Heating

Признак **Heating** указывает на наличие отопления. Посмотрим на него:

In [178]:
#уберем ненужные ковычки
df['Heating'] = df['Heating'].str.replace("'",'', regex=True)
df['Heating'].unique()[:10]

array(['Central A/C, Heat Pump', '', 'Forced Air', 'Electric, Heat Pump',
       'Gas', 'Central Electric', 'Forced air', 'Other',
       'Forced air, Heat pump', 'Central Air'], dtype=object)

In [179]:
df['Heating'] = df['Heating'].str.lower()
df['Heating'].value_counts()

forced air                                                      134308
                                                                105760
other                                                            29622
electric                                                         10215
gas                                                               9296
                                                                 ...  
zoned heating, wall unit heating, forced air heating                 1
baseboard, spacewallunit                                             1
hot air, stove-pellet                                                1
natural gas, space heater                                            1
baseboard, hot water, programmable thermostat, radiant floor         1
Name: Heating, Length: 1918, dtype: int64

В признаке содержится информация о типах отопления, отапливаемых участках и т.д. Предположим, что отсутсвующие значения указвают на отсутсвие отопления. Булево закодируем признак, наличие отопления будет *True*, отсутвтие -  *False*:

In [180]:
df['Heating'] = df['Heating'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none'] else False)

In [181]:
df['Heating'].value_counts()

True     259159
False    117975
Name: Heating, dtype: int64

#### Признак Cooling

Признак **Cooling** указывает на наличие системы кондиционирования, кондиционера. Посмотрим на него:

In [182]:
#уберем ненужные ковычки
df['Cooling'] = df['Cooling'].str.replace("'",'', regex=True)
df['Cooling'].unique()[:10]

array(['', 'Central', 'Central Air', 'Central Gas', 'Steam Heating',
       'Central, Evaporative', 'Has Cooling',
       'Air Conditioning Unit(s), Ceiling Fan(s)', 'Central A/C',
       'No Data'], dtype=object)

In [183]:
df['Cooling'] = df['Cooling'].str.lower()
df['Cooling'].value_counts()

central                                                                158744
                                                                       120395
central air                                                             14384
none                                                                    10923
no data                                                                 10615
                                                                        ...  
central gas, propane, zoned                                                 1
other (see remarks), panel/floor/wall, window unit                          1
multi units, zoned cooling                                                  1
central air, g-energy star hvac, gas hot air/furnace, multizone a/c         1
central a/c (gas), central heat (gas), heat pump                            1
Name: Cooling, Length: 1438, dtype: int64

В признаке содержится информация о типах кондиционирования. Предположим, что отсутсвующие значения указвают на отсутсвие кондиционера, системы кондиционирования. Булево закодируем признак, наличие - *True*, отсутвтие -  *False*:

In [184]:
df['Cooling'] = df['Cooling'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none'] else False)

In [185]:
df['Cooling'].value_counts()

True     235201
False    141933
Name: Cooling, dtype: int64

#### Признак Parking

Признак **Parking** указывает на наличие парковки. Посмотрим на него:

In [186]:
#уберем ненужные ковычки
df['Parking'] = df['Parking'].str.replace("'",'', regex=True)
df['Parking'].unique()[:10]

array(['', 'Attached Garage', 'Detached Garage',
       'Carport, Attached Garage', '2 spaces', '4 spaces', 'Off Street',
       'None', '0', '1 space'], dtype=object)

In [187]:
df['Parking'] = df['Parking'].str.lower()
df['Parking'].value_counts()

                                                                                                                                    171846
attached garage                                                                                                                      70748
2 spaces                                                                                                                             28061
1 space                                                                                                                              14252
no data                                                                                                                              13333
                                                                                                                                     ...  
assigned, open, under building                                                                                                           1
assigned parking space - 1,

В признаке содержится информация о типах парковки, количестве парковочных мест и т.д. Предположим, что отсутсвующие значения указвают на отсутсвие парковки, парковочного местка. Булево закодируем признак, наличие - *True*, отсутвтие -  *False*:

In [188]:
df['Parking'] = df['Parking'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none','0'] else False)

In [189]:
df['Parking'].value_counts()

False    192194
True     184940
Name: Parking, dtype: int64

####  Признак lotsize

Признак **lotsize** указывает на размер продаваемого лота. Посмотрим на него:

In [190]:
#уберем ненужные ковычки
df['lotsize'] = df['lotsize'].str.replace("'",'', regex=True)
df['lotsize'].unique()[:10]

array(['None', '5828 sqft', '8,626 sqft', '8,220 sqft', '10,019 sqft',
       '680 sqft', '4,996 Sq. Ft.', '8,750 Sq. Ft.', '124582',
       '2,056 sqft'], dtype=object)

In [191]:
df['lotsize'] = df['lotsize'].str.lower()
df['lotsize'].value_counts()

                  33055
none              28387
—                 25251
no data            5329
-- sqft lot        3818
                  ...  
43160                 1
8,758 sq. ft.         1
8,511 sqft lot        1
3400 sqft             1
7,084 sqft lot        1
Name: lotsize, Length: 36609, dtype: int64

In [192]:
l = list(df['lotsize'].unique())
l

['none',
 '5828 sqft',
 '8,626 sqft',
 '8,220 sqft',
 '10,019 sqft',
 '680 sqft',
 '4,996 sq. ft.',
 '8,750 sq. ft.',
 '124582',
 '2,056 sqft',
 '1.73 acres',
 '5,715 sq. ft.',
 '10,270 sqft',
 '7,000 sqft lot',
 '2,130 sqft',
 '2,003 sq. ft.',
 '5,304 sq. ft.',
 '1,782 sq. ft.',
 '10,424 sqft',
 '8,110 sqft',
 '1.18 acres',
 '15.32 acres lot',
 '—',
 '7962 sqft',
 '8,120 sqft',
 '2,080 sqft',
 '3828 sqft',
 '9,000 sqft',
 '5,249 sqft',
 '4828',
 '8,189 sqft lot',
 '0.28 acres',
 '7,782 sqft',
 '',
 '7,440 sqft',
 '5,998 sqft',
 '2.14 acres lot',
 '9.25 acres',
 '0.25 acres',
 '2,052 sqft',
 '7497',
 '0.91 acres lot',
 '7,218 sqft',
 '50530',
 '10018 sqft',
 '7,000 sqft',
 '6,251 sqft',
 '110207',
 '6,000 sqft',
 '9,600 sqft',
 '0.29 acres',
 '1.7 acres',
 'no data',
 '11920',
 '4,696 sqft',
 '0.62 acres',
 '7,405 sqft',
 '4983',
 '7,680 sqft',
 '10,200 sq. ft.',
 '25.41 acres',
 '5,227 sqft lot',
 '103237',
 '9,263 sqft',
 '654 sqft',
 '0.36 acres',
 '5,623 sqft',
 '-- sqft lot',
 '1.

В признаке большое количество пропусков. Удалим его:

In [193]:
df = df.drop('lotsize', axis=1)

#### Признак Price/sqft

Признак **Price/sqft** показывает отношение цены к площади и как следствие зависим от целевого признака. Во избежания утечки данных удалим его:

In [194]:
df= df.drop('Price/sqft', axis=1)

### MlsId и mls-id

Параметры **MlsId** и **mls-id** -  идентификаторы система мультилистинга, содержат уникальный идентификационный номер объекта. Параметр не влияет на стоимость объекта, его следует удалить:

In [195]:
df = df.drop(['MlsId', 'mls-id'], axis=1)

### Признак target

Признак **target** указывает стоимость объекта и является нашим целевый признаков. Посмотрим на него:

In [196]:
print(f"Количество пропущенных значений в признаке target: {df['target'].isna().sum()}\n"
    f"Отношение пропущенных значений признака target к объему наблюдений: {(round(df['target'].isna().sum()/len(df)*100,2))}%\n"
    f"Количество уникальных значений признака target: {df['target'].nunique()}\n"
    f"Уникальные значения признака target: {list(df['target'].sort_values().unique()[:10])}")

Количество пропущенных значений в признаке target: 2480
Отношение пропущенных значений признака target к объему наблюдений: 0.66%
Количество уникальных значений признака target: 43939
Уникальные значения признака target: ['$1', '$1,000', '$1,000,000', '$1,000,000+', '$1,000,036', '$1,000,050', '$1,000,100', '$1,000,898', '$1,000/mo', '$1,001,713']


Удалим пропуски:

In [197]:
df = df.dropna(subset=['target'])

В признаке присутствуют значения с приставков /mo указывающей на помесячную оплату. Наша задача построить модель предсказывающую стоимость недвижимости, данные по аренде будут давать искажения - их лучше удалить. Посмотрим на долю данных по аренде и соотнесем их с данными с приставков /mo:

In [198]:
l = list(df['status'].unique())
l

['auction',
 'for sale',
 'No info',
 'new construction',
 'new',
 'pending',
 'active/contingent',
 'Under Contract',
 'pre-foreclosure / auction',
 'Under Contract Showing',
 'Pre-foreclosure',
 'Under Contract Backups',
 'active under contract',
 'foreclosure',
 'foreclosed',
 'for rent',
 'contingent',
 'pending Continue To Show',
 'price change',
 'back on market',
 'active option',
 'Coming soon: Nov',
 'contingent finance and inspection',
 'Coming soon: Dec',
 'Pending Sale',
 'active with contingencies',
 'contingent show',
 'contract p',
 'contingent take backup',
 'apartment for rent',
 'backup contract',
 'option contract',
 'Pending Inspection',
 'active option contract',
 'closed',
 'contingent   show',
 'due diligence period',
 'contract contingent on buyer sale',
 'u under contract',
 'active contingency',
 'Pending With Contingencies',
 'lease/purchase pending',
 'Pending Taking Backups',
 'c continue show',
 'offer pending signature',
 'contingent lien holder release',

In [199]:
rent_df = df[(df['status'] == 'for rent') | (df['status'] == 'apartment for rent') | (df['status'] == 'condo for rent')]
matching_rows = df[df.target.str.contains('/mo',regex=True)]
print(f"Разница числа строк со статусом аренды и числа строк с указанием помесячной оплаты:{len(rent_df) - len(matching_rows)}")

Разница числа строк со статусом аренды и числа строк с указанием помесячной оплаты:14


Видим что если произвести удаление строк по окончанию /mo в **target**, то останется еще 14 строк связаных с арендой, удалим данные по аренде через признак **status**:

In [200]:
df = df[(df['status'] != 'for rent') & (df['status'] != 'apartment for rent') & (df['status'] != 'condo for rent')]

Оставим в признаке только числовые значения и переведем к целочисленному типу:

In [201]:
df.loc[:,'target'] = df.loc[:,'target'].str.replace('[^0-9]','',regex=True)
df.target = df.target.astype(int)

Посмотрим на разброс значений в целевом признаке:

In [202]:
display(df[df['target'] == df['target']. max ()].head())
display(df[df['target'] == df['target']. min ()].head())


,status,propertyType,street,baths,fireplace,city,schools,sqft,zipcode,state,PrivatePool,target,Year built,Heating,Cooling,Parking
132416,for sale,single_family_home,875 nimes rd,18.0,True,los angeles,"[{'rating': ['10/10', '7/10'], 'data': {'Dista...",0,90077,CA,True,195000000,1935,True,True,True


,status,propertyType,street,baths,fireplace,city,schools,sqft,zipcode,state,PrivatePool,target,Year built,Heating,Cooling,Parking
15533,auction,single_family_home,1230 old dickerson rd,2.0,False,goodlettsville,"[{'rating': ['3', '2', '3', '3', '2', '1', 'NR...",1596,37072,TN,False,1,1940,True,False,True
25334,No info,single_family_home,3205 n high school rd,1.0,True,indianapolis,"[{'rating': ['2/10', 'NA', '1/10'], 'data': {'...",1586,46224,IN,False,1,1957,True,True,True
84946,foreclosure,land,7804 hamilton ave,0.0,True,cincinnati,"[{'rating': ['2/10'], 'data': {'Distance': ['0...",2536,45231,OH,False,1,1865,False,True,True
148230,auction,land,cummings st,0.0,False,huntley,"[{'rating': [], 'data': {'Distance': [], 'Grad...",0,60142,IL,False,1,no info,False,False,False
205341,auction,land,route 47 hwy,0.0,False,huntley,"[{'rating': ['8', '9', '8', '7', '7', 'NR'], '...",0,60142,IL,False,1,no info,False,False,False


Видим наличие выбросов. Примим за минимально допустимую стоимость 1000 т.к. у нас есть объекты с лешением права выкупа и акуционном, 1000 вполне может быть стартовой стоимостью. Удалим все значания ниже 1000:

In [203]:
df = df.drop(df[df['target'] < 1000].index)

In [204]:
print(f"Количество пропущенных значений в признаке target: {df['target'].isna().sum()}\n"
    f"Отношение пропущенных значений признака target к объему наблюдений: {(round(df['target'].isna().sum()/len(df)*100,2))}%\n"
    f"Количество уникальных значений признака target: {df['target'].nunique()}\n"
    f"Уникальные значения признака target: {list(df['target'].sort_values().unique()[:10])}")

Количество пропущенных значений в признаке target: 0
Отношение пропущенных значений признака target к объему наблюдений: 0.0%
Количество уникальных значений признака target: 34162
Уникальные значения признака target: [1000, 1100, 1200, 1299, 1300, 1325, 1350, 1425, 1450, 1500]


### Признак schools

Признак **schools** содержит информацию о школах в районе. Посмотрим на признак:

In [205]:
print(f"Количество пропущенных значений в признаке schools: {df['schools'].isna().sum()}\n"
    f"Отношение пропущенных значений признака schools к объему наблюдений: {(round(df['schools'].isna().sum()/len(df)*100,2))}%\n"
    f"Количество уникальных значений признака schools: {df['schools'].nunique()}\n"
    f"Уникальные значения признака schools: {list(df['schools'].sort_values().unique()[:10])}")

Количество пропущенных значений в признаке schools: 0
Отношение пропущенных значений признака schools к объему наблюдений: 0.0%
Количество уникальных значений признака schools: 294995
Уникальные значения признака schools: ["[{'rating': [''], 'data': {'Distance': [], 'Grades': []}, 'name': []}]", "[{'rating': ['1', '1', '1', '1', '2', '5', 'NR', 'NR'], 'data': {'Distance': ['0.8 mi', '0.5 mi', '0.8 mi', '1.1 mi', '0.5 mi', '0.5 mi', '0.3 mi', '0.3 mi'], 'Grades': ['K–5', '6–8', '9–12', 'K–5', 'K–12', '9–12', 'K–8', 'K–8']}, 'name': ['Forty-Second Street Elementary School', 'Audubon Middle School', 'Crenshaw Science, Technology, Engineering, Math And Medicine Magnet School', 'Fifty-Fourth Street Elementary School', 'Marlton School', 'Crenshaw Arts-Technology Charter High School', 'Career Elementary Preparatory Academy Private School', 'Fortitude CCS Private School']}]", "[{'rating': ['1', '1', '1', '1', '2', 'NR', 'NR', 'NR'], 'data': {'Distance': ['0.1 mi', '0.9 mi', '0.5 mi', '0.4 mi',

In [206]:
l = list(df['schools'].sort_values().unique())
l

["[{'rating': [''], 'data': {'Distance': [], 'Grades': []}, 'name': []}]",
 "[{'rating': ['1', '1', '1', '1', '2', '5', 'NR', 'NR'], 'data': {'Distance': ['0.8 mi', '0.5 mi', '0.8 mi', '1.1 mi', '0.5 mi', '0.5 mi', '0.3 mi', '0.3 mi'], 'Grades': ['K–5', '6–8', '9–12', 'K–5', 'K–12', '9–12', 'K–8', 'K–8']}, 'name': ['Forty-Second Street Elementary School', 'Audubon Middle School', 'Crenshaw Science, Technology, Engineering, Math And Medicine Magnet School', 'Fifty-Fourth Street Elementary School', 'Marlton School', 'Crenshaw Arts-Technology Charter High School', 'Career Elementary Preparatory Academy Private School', 'Fortitude CCS Private School']}]",
 "[{'rating': ['1', '1', '1', '1', '2', 'NR', 'NR', 'NR'], 'data': {'Distance': ['0.1 mi', '0.9 mi', '0.5 mi', '0.4 mi', '0.5 mi', '0.5 mi', '0.5 mi', '0.5 mi'], 'Grades': ['K–5', '6–8', '9–12', 'K–5', 'K–12', '9–12', 'K–6', 'K–8']}, 'name': ['Coliseum Street Elementary School', 'Audubon Middle School', 'Susan Miller Dorsey Senior High Sc

Признак содержит сведения о наименование и рейтинге школы, растоянии до школы, оценки. Интерес для нас представляют растояние до школы и ее рейтинг. Приведем рейтинг к среднему значению, сформируем два новых признака: **school_rating _mean** - средний рейтинг школы, **school_dist_min** - минимальное растояние до школы. После удалим признак **schools**:

In [207]:
#Рассчитаем средний рейтинг
def process_rating(rating):
    rating = rating[0].replace('/10', '')
    extracted_numbers = [float(num) for num in rating if num.isdigit()]
    average_rating = np.average(extracted_numbers) if extracted_numbers else -1
    return round(average_rating, 1)

s_rating = (
    df.schools.str.findall(r"\brating': ([\s\S]+?), 'data\b")
    .apply(lambda x: process_rating(x))
)

#Создадим признак school_rating _mean в нашем наборе данных. Пропуски заполним -1 
df['school_rating _mean'] = s_rating

In [208]:
#Извлечение расстояний до школ из каждой строки
s_distance = df.schools.str.findall(r"\bDistance': ([\s\S]+?), 'Grades\b") 
#Получение первого найденного значения списков, полученных в результате предыдущего шага
s_distance = s_distance.apply(lambda x: x[0]) 
#Удаление всех буквенных символов из расстояний
s_distance = s_distance.str.replace('[a-zA-Z]','', regex=True) 
#Извлечение чисел, включая десятичные значения
s_distance = s_distance.str.findall(r'\b([0-9]+.[0-9]+)') 
#Преобразуем значения списка в тип float
s_distance = s_distance.apply(lambda x: [float(i) for i in x]) 
# Нахождение минимального значения в каждом списке s_distance, и возвращение -1, если длина списка равна 0
school_dist_min = s_distance.apply(lambda x: -1 if len(x)==0 else min(x)) 


#Создадим признак school_dist_min в нашем наборе данных. Пропуски заполним -1
df['school_dist_min'] = school_dist_min

Удалим признак **schools**:

In [209]:
df = df.drop('schools', axis=1)

## 4. ПРОВЕРЯЕМ И ДОЧИЩАЕМ ДАННЫЕ ПОСЛЕ ПРЕОБРАЗОВАНИЯ И ОЧИСТКИ ПРИЗНАКОВ

Посмотрим на количество дубликатов в данных:

In [210]:
df.duplicated().sum() 

414

Удаляем дубликаты:

In [211]:
df = df.drop_duplicates(ignore_index=True)

Строки с отсутсвующией информацией по признакам **baths**, **sqft**, **Year built** представляются малоинформативными. Посмотрим на строки содержащие пропуски по трем признакам **baths**, **sqft**, **Year built**:

In [212]:
result = df[(df['baths'] == 0) & (df['sqft'] == 0) & (df['Year built'] == 'no info')]

print(f"Количество строк с пропусками в признаках baths, sqft, Year built:{len(result)}")
print(f"Отношение строк с пропусками в признаках baths, sqft, Year built ко всему объему данных:{round((len(result) / len(df) * 100),2)}%")

Количество строк с пропусками в признаках baths, sqft, Year built:41903
Отношение строк с пропусками в признаках baths, sqft, Year built ко всему объему данных:11.21%


Таких строк у нас всего 11.21%. Удалим их:

In [213]:
df = df[~((df['baths'] == 0) & (df['sqft'] == 0) & (df['Year built'] == 'no date'))]


Посмотрим на данные после всех приобразований:

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 373752 entries, 0 to 373751
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               373752 non-null  object 
 1   propertyType         373752 non-null  object 
 2   street               373752 non-null  object 
 3   baths                373752 non-null  float64
 4   fireplace            373752 non-null  bool   
 5   city                 373752 non-null  object 
 6   sqft                 373752 non-null  int32  
 7   zipcode              373752 non-null  object 
 8   state                373752 non-null  object 
 9   PrivatePool          373752 non-null  bool   
 10  target               373752 non-null  int32  
 11  Year built           373752 non-null  object 
 12  Heating              373752 non-null  bool   
 13  Cooling              373752 non-null  bool   
 14  Parking              373752 non-null  bool   
 15  school_rating _me

Данные очищены и приобразованны. Сохраним новый датафрейм без создания нового индекса:

In [215]:
df.to_csv("data/cleaned_data.csv", index=False)